In [5]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import parse_UK_Data, generate_large_distance_matrix, load_saved_here_data, save_here_data, get_results


dataset = "../datasets/UK10_1.txt"
meta_data, distance_data, station_data = parse_UK_Data(dataset)
station_coordinates = pd.read_csv('../datasets/UK10_1_coordinates.csv', encoding='utf-8')
points_coordinate = station_coordinates[["lat", "long"]].to_numpy()

api = "Tfmt6Sxyn-nfinSHUyUo3xmVDWdtwAUyh6C8Y3HTZhU"

route = [[0,1,2,3], [4,5,6,7,8,9]]

In [40]:
meta_data["F-C Empty (l/100km)"] = 12.5
meta_data["F-C Full (l/100km)"] = 15
meta_data["Vehicle Weight"] = 3000
#distance_matrix_here,time_matrix_here,_ = generate_large_distance_matrix(points_coordinate,api)
#save_here_data("../datasets/", "UK10_1", distance_matrix_here, time_matrix_here)
distance_matrix_here,time_matrix_here = load_saved_here_data("../datasets/",dataset)

In [49]:
def generate_final_results(results:list, algorithm_meta_data:list):
    
    # Visa Max,Min,Avg av alla parametrar
    # En algortims körs 3ggr
    # Visa även algortimens beräkningstid
    # Visa grafer över prestanda
    
    data_df = pd.concat(results, axis=1)
#     result_df = pd.concat({"Max":data_df.max(axis=0),
#                            "Min":data_df.min(axis=0),
#                            "Mean":data_df.mean(axis=0),
#                            "Std":data_df.std(axis=0)}, axis=1)
    
    return data_df

data1 = get_results(route, distance_matrix=distance_matrix_here, demand_data=station_data, meta_data=meta_data, travel_time_matrix=time_matrix_here)
data2 = get_results(route, distance_matrix=distance_matrix_here, demand_data=station_data, meta_data=meta_data, travel_time_matrix=time_matrix_here)

generate_final_results([data1, data2], [])

,Total distance (km),Total load (kg),Total Estimated Fuel Consumption (L) (Hao et al.),Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.),Total Estimated Fuel Consumption (L) (Rakha et al.),Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.),Avg Speed (km/h),Total Travel Time (s),Travel Time hh:mm:ss,Total distance (km),Total load (kg),Total Estimated Fuel Consumption (L) (Hao et al.),Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.),Total Estimated Fuel Consumption (L) (Rakha et al.),Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.),Avg Speed (km/h),Total Travel Time (s),Travel Time hh:mm:ss
0,352.582,1535,46.399187,1.315983,34.590115,0.981052,74.462936,17046,04:44:06.00,352.582,1535,46.399187,1.315983,34.590115,0.981052,74.462936,17046,04:44:06.00
1,943.246,2322,124.497266,1.319881,92.208960,0.977571,74.943403,45310,12:35:10.00,943.246,2322,124.497266,1.319881,92.208960,0.977571,74.943403,45310,12:35:10.00


,Total distance (km),Total load (kg),Total Estimated Fuel Consumption (L) (Hao et al.),Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.),Total Estimated Fuel Consumption (L) (Rakha et al.),Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.),Avg Speed (km/h),Total Travel Time (s),Travel Time hh:mm:ss
0,352.582,1535,46.399187,1.315983,34.590115,0.981052,74.462936,17046,04:44:06.00
1,943.246,2322,124.497266,1.319881,92.208960,0.977571,74.943403,45310,12:35:10.00
